In [6]:
%pip install instagrapi python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: instagrapi in c:\users\dell\appdata\local\programs\python\python311\lib\site-packages (2.1.3)
                                              0.0/11.6 MB ? eta -:--:--
     -                                        0.4/11.6 MB 11.6 MB/s eta 0:00:01
     -                                        0.6/11.6 MB 8.8 MB/s eta 0:00:02
     --                                       0.7/11.6 MB 6.3 MB/s eta 0:00:02
     ---                                      0.9/11.6 MB 5.6 MB/s eta 0:00:02
     ---                                      1.1/11.6 MB 5.3 MB/s eta 0:00:02
     ----                                     1.2/11.6 MB 5.1 MB/s eta 0:00:03
     ----                                     1.3/11.6 MB 4.4 MB/s eta 0:00:03
     ----                                     1.4/11.6 MB 4.1 MB/s eta 0:00:03
     ----                                     1.4/11.6 MB 4.1 MB/s eta 0:00:03
     ----                 

In [3]:
%load_ext dotenv
%dotenv

In [60]:
from instagrapi import Client
import os
import pandas as pd
import json

In [20]:
ACCOUNT_USERNAME = os.environ.get("ACCOUNT_USERNAME")
ACCOUNT_PASSWORD = os.environ.get("ACCOUNT_PASSWORD")

cl = Client()
cl.login(ACCOUNT_USERNAME, ACCOUNT_PASSWORD)

True

In [44]:
df = pd.DataFrame()

def collect_data(username):
    global df
    user = cl.user_info_by_username_v1(username)
    raw_medias = cl.user_medias(user.pk)
    media = [x.model_dump() for x in raw_medias]
    media = pd.DataFrame({k: [x[k] for x in media] for k in media[0]})
    df = pd.concat([df, media])

In [45]:
users = [
    "beerbiceps",
    "hiteshchoudharyofficial",
    "levelsupermind",
    "neurotechh",
    "mrbeast",
    "433",
    "khaby00",
    "viraj_ghelani",
    "indiasgotlatent",
]

for username in users:
    collect_data(username)
print(len(df))

4331


In [107]:
id_to_username = {}

for username in users:
    id_to_username[cl.user_info_by_username_v1(username).pk] = username

print(json.loads(df["user"][0].replace("'", '"')))

df["username"] = df.apply(
    lambda x: json.loads(x.user.replace("'", '"'))["username"], axis=1
)
df["location_name"] = df.apply(
    lambda x: x.location.get("name") if x.location else "", axis=1
)

'{"pk": "501113371", "username": "beerbiceps", "full_name": "Ranveer Allahbadia", "profile_pic_url": HttpUrl("https://instagram.fbom19-3.fna.fbcdn.net/v/t51.2885-19/418671390_1098365544837568_3016557017977007907_n.jpg?stp=dst-jpg_e0_s150x150_tt6&_nc_ht=instagram.fbom19-3.fna.fbcdn.net&_nc_cat=1&_nc_ohc=JW2pD15fp2AQ7kNvgEM0mR3&_nc_gid=e72647a88cb14a62ae7729830b5f03ce&edm=ABmJApABAAAA&ccb=7-5&oh=00_AYAuWU_WNqXgKe7jkDf7-UvMtmq4ayAkuCjEFGJsJl_q5g&oe=677AA64C&_nc_sid=b41fef"), "profile_pic_url_hd": None, "is_private": False}'

In [113]:
df["caption_text"] = df.apply(lambda x: str(x["caption_text"]).strip(), axis=1)
df = df.drop(columns=["id", "comments_disabled", "commenting_disabled_for_viewer", "has_liked", "accessibility_caption", "title", "thumbnail_url"])

In [114]:
df.to_csv("dataset.csv", index=False)